# $k$ Nearest Neighbors

## Illustration of Nearest Neighbors

Arguably one of the simplest classification algorithm is $k$ nearest neighbors (KNN). Below figure portrays the algorithm's simplicity. We see a training set of five red and five blue dots, representing some label 1 and 0, respectively. The two axes represent two features, e.g. income and credit card balance. If we now add a new test data point $x_0$ (green dot), KNN will label this test data according to its $k$ closest neighbors. In below figure we set $k=3$. The three closest neighbors are: one blue dot and two red dots, resulting in estimated probabilities of 2/3 for the red class and 1/3 for the blue class. Hence KNN will predict that the new data point will belong to the red class. More so, based on the training set the algorithm is able to draw a decision boundary. (Note: In a classification settings with $j$ classes a *decision boundary* is a hyperplane that partitions the underlying vector space into $j$ sets, one for each class.). This is shown with the jagged line that separates background colors cyan (blue label) and light blue (red label). Given any possible pair of feature values, KNN labels the response along the drawn decision boundary. With $k=1$ the boundary line is very jagged. Increasing the number of $k$ will smoothen the decision boundary. This tells us that small values of $k$ will produce large variance but low bias, meaning that each new added training point might change the decision boundary line significantly but the decision boundary separates the training set (almost) correctly. As $k$ increases, variance decreases but bias increases. This is a manifestation of the Bias-Variance Trade-Off as discussed in the script ([Fortmann-Roe (2012)](http://scott.fortmann-roe.com/docs/BiasVariance.html)) and highlights the importance of selecting an adequate value of $k$ - a topic we will pick up in a future chapter.

<img src="Graphics/0207_KNN.png" alt="LinearRegClassification" style="width: 1000px;"/>

## Mathematical Description of KNN

Having introduced KNN illustratively, let us now define this in mathematical terms. Let our data set be $\{(x_1, y_1), (x_2, y_2), \ldots (x_n, y_n)\}$ with $x_i \in \mathbb{R}^p$ and $y_i \in \{0, 1\}\; \forall i \in \{1, 2, \ldots, n\}$. Based on the $k$ neighbors, KNN estimates the conditional probability for class $j$ as the fraction of points in $\mathcal{N}(k, x_0)$ (the set of the $k$ closest neighbors of $x_0$) whose response values equals $j$ (Russell and Norvig (2009)):

$$\begin{equation}
\Pr(Y = j | X = x_0) = \frac{1}{k} \sum_{i \in \mathcal{N}(k, x_0)} \mathbb{I}(y_i = j).
\end{equation}$$

Once the probability for each class $j$ is calculated, the  KNN classifier predicts a class label $\hat{y}_0$ for the new data point $x_0$ by maximizing the conditional probability (Batista and Silva (2009)).

$$\begin{equation}
\hat{y}_0 = \arg \max_{j} \frac{1}{k} \sum_{i \in \mathcal{N}(k,x_0)} \mathbb{I}(y_i = j)
\end{equation}$$

Selecting a new data point $x_0$'s nearest neighbors requires some notion of **distance measure**. Most researchers chose Minkowski's distance, which is often referred to as $L^m$ norm (Guggenbuehler (2015)). The distance between points $x_a$ and $x_b$ in $\mathbb{R}^p$ is then defined as follows (Russell and Norvig (2009)):

$$\begin{equation}
L^m (x_a, x_b) = \left(\sum_{i=1}^p |x_{a, i} - x_{b, i}|^m \right)^{1/m}
\end{equation}$$

Using $m=2$, above equation simplifies to the well known Euclidean distance and $m=1$ yields the Manhattten distance. Python's `sklearn` package, short for scikit-learn, offers [several other options](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) that we will not discuss. 

Beyond the pure distance measure, it is also possible to weight training data points relative to their distance from a certain point. In above figure distance is weighted uniformly. Alternatively one could weight points by the inverse of their distance (closer neighbors of a query point will have a greater influence than neighbors which are further away) or any other user-defined weighting function. For [further details check `sklearn`'s documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html?highlight=neighbors%20kneighborsclassifier#sklearn.neighbors.KNeighborsClassifier) for details).

## Application: Predicting Share Price Movement

### Loading Data

The application of KNN is shown using simple stock market data. The idea is to predict a stock's movement based on simple features such as:
* `Lag1, Lag2`: log returns of the two previous trading days 
* `SMI`: SMI log return of the previous day

The response is a binary variable: if a stock closed above the previous day's closing price it equals 1, and 0 if it fell. We start by loading the necessary packages and stock data from a csv - a procedure we are well acquainted with by now. Thus comments are held short.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.size'] = 14

In [2]:
# Import daily shares prices and select window
shsPr = pd.read_csv('Data/SMIDataDaily.csv', sep=',',
                     parse_dates=['Date'], dayfirst=True,
                     index_col=['Date'])
shsPr = shsPr['2012-06-01':'2017-06-30']
shsPr = shsPr.sort_index()
shsPr.tail()

,ABBN,ADEN,BAER,CFR,CSGN,GEBN,GIVN,LHN,LONN,NESN,...,ROG,SCMN,SGSN,SLHN,SREN,UBSG,UHR,ZURN,SIK,SMI
Date,,,,,,,,,,,,,,,,,,,,,
2017-06-26,24.63,73.85,51.35,80.45,13.49,457.0,1947.0,56.25,208.2,85.65,...,251.7,469.3,2375.0,323.9,89.40,15.71,367.5,284.4,6310.0,9121.22
2017-06-27,24.40,73.70,51.65,80.20,13.74,453.8,1931.0,56.40,207.6,84.30,...,252.0,465.0,2355.0,322.7,88.80,16.00,362.0,281.7,6280.0,9072.92
2017-06-28,24.07,73.55,51.65,80.00,13.82,452.5,1927.0,56.35,207.5,85.40,...,251.2,464.9,2342.0,325.3,88.55,16.29,360.0,278.0,6195.0,9076.73
2017-06-29,23.63,72.85,51.30,78.60,14.10,446.0,1913.0,55.00,204.7,83.65,...,246.9,463.9,2310.0,323.3,87.80,16.36,353.4,276.3,6145.0,8944.04
2017-06-30,23.68,72.90,50.45,79.00,13.86,447.2,1918.0,54.90,207.3,83.45,...,244.2,462.7,2322.0,323.6,87.65,16.24,354.1,279.1,6160.0,8906.89


Having the data in a proper dataframe, we are now in a position to create the features and response values.

In [3]:
# Calculate log-returns and label responses: 
# 'direction' equals 1 if stock closed above 
# previous day and 0 if it fell.
today = np.log(shsPr / shsPr.shift(1))
direction = np.where(today >= 0, 1, 0)

# Convert 'direction' to dataframe
direction = pd.DataFrame(direction, index=today.index, columns=today.columns)

# Lag1, 2: t-1 and t-2 returns; excl. smi (in last column)
Lag1 = np.log(shsPr.iloc[:, :-1].shift(1) / shsPr.iloc[:, :-1].shift(2))
Lag2 = np.log(shsPr.iloc[:, :-1].shift(2) / shsPr.iloc[:, :-1].shift(3))

# Previous day return for SMI index
smi  = np.log(shsPr.iloc[:, -1].shift(1) / shsPr.iloc[:, -1].shift(2))

### KNN Algorithm Applied

Now comes the difficult part. What we want to achieve is to run the KNN algorithm for every stock and for different hyperparameter $k$ and see how it performs. For this we do the following steps:

1. Create a feature matrix `X` containing `Lag1`, `Lag2` and `SMI` data for share i
2. Create a response vector `y` with binary direction values
3. Split data to training (before 2016-06-30) and test set (after 2016-06-30)
4. Run KNN for different values of $k$ (loop)
5. Write test score for given $k$ to matrix `scr`
6. Once we have run through all $k$'s we proceed with step 1. with share i+1

This means we need two loops. The first corresponds to the share (e.g. ABB, Adecco, etc.), the second runs the KNN algorithm for different values of $k$. 

The reason for this approach is that we are interested in finding any pattern/structure that would provide a successful trading strategy. There is obviously no free lunch. Predicting share price direction is by no means an easy task and we must be well aware that we are in for a difficult job here. If it were simple, neither one of us would be sitting here but run his own fund. But nonetheless, let us see how KNN performs and how homogeneous (or heterogeneous) the results are.

As usual our first step is to prepare the ground by loading the necessary package and defining some auxiliary variables. The KNN function we will be using is available through the `sklearn` (short for Scikit-learn) package. We only load the `neighbor` sublibrary which contains the needed KNN function called `KNeigborsClassifier()`. KNN is applied with the default distance metric: Euclidean distance (Minkowski's distance with $m=2$). If we would prefer another distance metric we would have to specify it ([see documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)).

In [4]:
# Import relevant functions
from sklearn import neighbors

# k = {1, 3, ..., 200}
k = np.arange(1, 200, 2)

# Array to store results in. Dimension is [k x m] 
# with m=20 for the 20 companies (excl. SMI)
scr = np.empty(shape=(len(k), len(shsPr.columns)-1))

In [ ]:
for i in range(len(shsPr.columns)-1):
    
    # 1) Create matrix with feature values of stock i
    X = pd.concat([Lag1.iloc[:, i], Lag2.iloc[:, i], smi], axis=1)
    X = X[3:]  # Drop first three rows with NaN (due to lag)
    
    # 2) Remove first three rows of response dataframe
    #    to have equal no. of rows for features and response
    y = direction.iloc[:, i]
    y = y[3:]
    
    # 3) Split data into training set...
    X_train = X[:'2016-06-30']
    y_train = y[:'2016-06-30']
    # ...and test set.
    X_test = X['2016-07-01':]
    y_test = y['2016-07-01':]
    
    # Convert responses to 1xN array (with .ravel() function)
    y_train = y_train.values.ravel()
    y_test  = y_test.values.ravel()
    
    for j in range(len(k)):
        
        # 4) Run KNN
        # Instantiate KNN class
        knn = neighbors.KNeighborsClassifier(n_neighbors=k[j])
        # Fit KNN classifier using training set
        knn = knn.fit(X_train, y_train)
        
        # 5) Extract test score for k[j]
        scr[j, i] = knn.score(X_test, y_test)

In [ ]:
# Convert data to pandas dataframe
tickers = shsPr.columns
scr = pd.DataFrame(scr, index=k, columns=tickers[:-1])

In [ ]:
scr.head()

### Results & Analysis

Now let's see the results in an overview. 

In [ ]:
scr.describe()

In [ ]:
scr.max().nlargest(5)

Following finance theory, returns should be distributed symmetrically. Thus the simplest guess would be to expect a share price to increase on 50% of the days and to decrease on the remaining 50%. Similar to guessing a coin flip, if we would guess an 'up' movement for every day, we obviously would - in the long run - be correct 50% of the times. This would make for a score of 50%. 

Looking in that light at the above summary, we see some very interesting results. For 10 out of 20 stocks KNN produces test scores of > 50% for even the 0.25th percentile. Let's plot the ones with the highest test-scores (ABBN, ZURN, NOVN, SIK) to see at what value of $k$ the best test-score is achieved.

In [ ]:
nms = ['ABBN', 'ZURN', 'NOVN', 'SIK']

plt.figure(figsize=(12, 8))
for col in nms:
    scr[col].plot(legend=True)
plt.axhline(0.50, c='k', ls='--');

For Zurich the peak is early (max. score around $k=60$) while the others peak later, i.e. with higher values of $k$. Furthermore, it seems interesting that for $k > 40$ test scores remained (barely) above 50%. If this is indeed a pattern we would have found a trading strategy, wouldn't we? 

To further assess our results we look into KNN's prediction of Givaudan's stock movements. For this we rerun our KNN classifier algorithm for ABBN as before.

In [ ]:
# 1) Create matrix with feature values of stock i
X = pd.concat([Lag1['ABBN'], Lag2['ABBN'], smi], axis=1)
X = X[3:]  # Drop first three rows with NaN (due to lag)

In [ ]:
# 2) Remove first three rows of response dataframe
#    to have equal no. of rows for features and response
y = direction['ABBN']
y = y[3:]

In [ ]:
# 3) Split data into training set...
X_train = X[:'2016-06-30']
y_train = y[:'2016-06-30']
# ...and test set.
X_test = X['2016-07-01':]
y_test = y['2016-07-01':]

# Convert responses to 1xN array (with .ravel() function)
y_train = y_train.values.ravel()
y_test  = y_test.values.ravel()

For GIVN the maximum score is reached where $k=145$. You can check this with the `scr['ABBN'].idxmax()` command, which provides the index of the maximum value of the selected column. In our case, the index is equivalent to the value of $k$. Thus we run KNN with $k=145$.

In [ ]:
scr['ABBN'].idxmax()

In [ ]:
# 4) Run KNN
# Instantiate KNN class for GIVN with k=145
knn = neighbors.KNeighborsClassifier(n_neighbors=145)
# Fit KNN classifier using training set
knn = knn.fit(X_train, y_train)

# 5) Extract test score for ABB
scr_ABBN = knn.score(X_test, y_test)
scr_ABBN

The score of 59.68% is the very same as what we have seen in above summary statistic. Nothing new so far. (Recall that the score is the total of correctly predicted outcomes.)

However, the alert reader should by now raise some questions regarding our assumption that 50% of the returns should have been positive. In the long run, this might be true. But our training sample contained only 1'017 records and of these 534 were positive. 

In [ ]:
# Percentage of 'up' days in training set
y_train.sum() / y_train.size

Therefore, if we would guess 'up' for every day of our test set and **given the distribution of classes in the test set is exactly as in our training set**, then we would predict the correct movement in 52.51% of the cases. So in that light, the predictive power of our KNN algorithm has to be put in perspective to the 52.51%.

In summary, our KNN algorithm has a score of 59.68%. Our best guess (based on the training set) would yield a score of 52.51%. This still displays that overall our KNN algorithm outperforms our best guess. Nonetheless, the margin is smaller than initially thought. 

### Confusion Matrix

There are more tools to assess the accuracy of an algorithm. We postpone the discussion of these tools to a later chapter and at this stage restrict ourselves to the discussion of a tool called "confusion matrix". 

A confusion matrix is a convenient way of displaying how our classifier performs. In binary classification (with e.g. response $y \in \{0, 1\}$) there are four prediction categories possible (Ting (2011)):

* **True positive**: True response value is 1, predicted value is 1 ("hit")
* **True negative**: True response value is 0, predicted value is 0 ("correct rejection")
* **False positive**: True response value is 0, predicted value is 1 ("False alarm", Type 1 error)
* **False negative**: True response value is 1, predicted value is 0 ("Miss", Type 2 error)

These information help us to understand how our (KNN) algorithm performed. There are different two ways of arranging confusion matrix. James et al. (2013) follow the convention that column labels indicate the true class label and rows the predicted response class. Others have it transposed such that column labels indicate predicted classes and row labels show true values. We will use the latter approach as it is more common.

<img src="Graphics/0207_ConfusionMatrixExplained.png" alt="ConfusionMatrixExplained" style="width: 800px;"/>

To run this in Python, we first predict the response value for each data entry in our test matrix `X_test`. Then we arrange the data in a suitable manner.

In [ ]:
# Predict 'up' (=1) or 'down' (=0) for test set
pred = knn.predict(X_test)

In [ ]:
# Store data in DataFrame
cfm = pd.DataFrame({'True direction': y_test,
                    'Predicted direction': pred})
cfm.replace(to_replace={0:'Down', 1:'Up'}, inplace=True)

# Arrange data to confusion matrix
print(cfm.groupby(['Predicted direction','True direction']) \
      .size().unstack('Predicted direction'))

As mentioned before, rows represent the true outcome and columns show what class KNN predicted. In 31 cases, the test set's true response was 'down' (in our case represented by 0) and KNN correctly predicted 'down'. 120 times KNN was correct in predicting an 'up' (=1) movement. 19 returns in the test set were positive but KNN predicted a negative return. And in 83 out of 253 cases KNN predicted an 'up' movement whereas in reality the stock price decreased. The KNN score of 59.68% for ABB is the sum of true positive and negative (31 + 120) in relation to the total number of predictions (253 = 31 + 19 + 83 + 120). The error rate is 1 - score or (19 + 83)/253.

Class-specific performance is also helpful to better understand results. The related terms are **sensitivity** and **specifity**. In the above case, sensitivity is the percentage of true 'up' movements that are identified. A good 86.3% (= 120 / (19 + 120)). The specifity is the percentage of 'down' movements that are correctly identified, here a poor 27.2% (= 31 / (31 + 83)). More on this in the next chapter.

Because confusion matrices are important to analyze results, `Scikit-learn` has its own [command to generate it](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). It is part of the `metrics` sublibrary. The difficulty is that in contrast to above (manually generated) table, the function's output provides no labels. Therefore one must be sure to know which value are where. Here's the code to generate the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion matrix
confusion_matrix(y_test, pred)

In general it is tremendously helpful to visualize results. At the time of writing in February 2022, Anaconda shipped with `Sklearn` version 0.24.2. This `Sklearn` version uses a confusion matrix-plotting function that is deprecated in versioun 1.0. Therefore, be aware that below plotting function only works for versions < 1.0. Details see [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html#sklearn.metrics.plot_confusion_matrix). 

In [ ]:
from sklearn.metrics import plot_confusion_matrix

# To plot the normalized figures, add normalize = 'true', 'pred', or 'all'
plot_confusion_matrix(estimator = knn, 
                      X = X_test, y_true = y_test,
                      display_labels = ['Down', 'Up'],
                      values_format = '.0f');

In [ ]:
# To plot the normalized figures, add normalize = 'true', 'pred', or 'all'
# This time with different color scheme
plot_confusion_matrix(estimator = knn, 
                      X = X_test, y_true = y_test,
                      cmap = plt.cm.Blues,
                      normalize = 'all',
                      display_labels = ['Down', 'Up'],
                      values_format = '.2f');

# Further Ressources


In writing this notebook, many ressources were consulted. For internet ressources the links are provided within the textflow above and will therefore not be listed again. Beyond these links, the following ressources were consulted and are recommended as further reading on the discussed topics:

* Batista, Gustavo, and Diego Furtado Silva, 2009, How k-nearest neighbor parameters affect its performance, in *Argentine Symposium on Artificial Intelligence*, 1–12, sn.
* Fortmann-Roe, Scott, 2012, Understanding the Bias-Variance Tradeoff from website, http://scott.fortmann-roe.com/docs/BiasVariance.html, 08/15/17.
* Guggenbuehler, Jan P., 2015, Predicting net new money using machine learning algorithms and newspaper articles, Technical report, University of Zurich, Zurich.
* James, Gareth, Daniela Witten, Trevor Hastie, and Robert Tibshirani, 2013, *An Introduction to Statistical Learning: With Applications in R* (Springer Science & Business Media, New York, NY).
* Müller, Andreas C., and Sarah Guido, 2017, *Introduction to Machine Learning with Python* (O’Reilly Media, Sebastopol, CA).
* Russell, Stuart, and Peter Norvig, 2009, *Artificial Intelligence: A Modern Approach* (Prentice Hall Press, Upper Saddle River, NJ).
* Ting, Kai Ming, 2011, Confusion matrix, in Claude Sammut, and Geoffrey I. Webb, eds., *Encyclopedia of Machine Learning* (Springer Science & Business Media, New York, NY).

In [ ]:
import sklearn
sklearn.show_versions()